In [1]:
def create_images_list(path):
    import os.path
    import glob
    import numpy as np
    from sklearn.cross_validation import train_test_split
    from tensorflow.python.platform import gfile
    
    images_cats = []
    labels_cats = []
    images_dogs = []
    labels_dogs = []
    
    extensions = ['jpg']
    file_list = []
    for extension in extensions:
        file_glob = os.path.join(path,'*.' + extension)
        file_list.extend(glob.glob(file_glob))
        
        if not file_list: continue
            
        for file_name in file_list:
            file_name_with_null = file_name.replace(path+"\\","")
            label = file_name_with_null.split(".")[0]
            if label == 'cat':
#                 cats = gfile.FastGFile(file_name, 'rb').read()
                images_cats.append(file_name)
                labels_cats.append(1)
            else:
#                 dogs = gfile.FastGFile(file_name, 'rb').read()
                images_dogs.append(file_name)
                labels_dogs.append(0)
    images = np.hstack((images_cats, images_dogs))
    labels = np.hstack((labels_cats, labels_dogs))
    print("laebls:", labels)
        
    num_images = len(labels)
    print("num_images:", num_images)
        
    input_data = np.array([images, labels])
    input_data = input_data.transpose()
    print("input_data:", input_data[0][1])
    
    np.random.shuffle(input_data)
        
    input_images = input_data[:, 0]
    input_labels = input_data[:, 1]
    input_labels = [int(i) for i in input_labels]
    
    print("input_labels:", input_labels)
    print("input_images:", input_images[0])
    
        
    train_images, valid_images, train_labels, valid_labels = train_test_split(input_images, input_labels, test_size=0.25, random_state=0)
        
    return train_images, train_labels, valid_images, valid_labels
    
# import matplotlib.pyplot as plt
# TRAIN_DATA_PATH = "C:/Users/yeyiy/Downloads/Dogs vs Cats/maybe"
# train_images, train_labels, valid_images, valid_labels = create_images_list(TRAIN_DATA_PATH)
# print(train_labels) 

In [2]:
def make_batch(images, labels, weight, height, batch_size):
    import numpy as np
    images = tf.cast(images, tf.string)
    labels = tf.cast(labels, tf.int32)
    
    input_data = tf.train.slice_input_producer([images, labels])
    
    image_raw = tf.read_file(input_data[0])
    labels = input_data[1]
    
    # 图像预处理开始
    images = tf.image.decode_jpeg(image_raw, channels=3)
    # 转换图像张量的类型
    images = tf.image.convert_image_dtype(images, dtype=tf.float32)
    # 调整图像为神经网络输入层的大小，调整算法随机
    images = tf.image.resize_images(images, [weight, height],
                                   method=np.random.randint(4))
    # 将图像进行随机左右反转
    images = tf.image.random_flip_left_right(images)
    images = tf.image.per_image_standardization(images)
    # 图像预处理完成
    print("images:", images)
    print("labels:", labels)
    
    min_after_dequeue = 2000
    capacity = min_after_dequeue + 3 * batch_size
    
    image_batch, label_batch = tf.train.batch([images, labels],
                                   batch_size = batch_size,
                                   num_threads = 64,
                                   capacity = capacity)
    
    label_batch = tf.reshape(label_batch, [batch_size])
    return image_batch, label_batch
    

In [3]:
def inference(image_batch, train):
    INPUT_NODE = 268203
    OUTPUT_NODE = 2
    
    IMAGE_SIZE = 299
    NUM_CHANNELS = 3
    NUM_LABELS = 2
    
    CONV1_SIZE = 5
    CONV1_DEEP = 32
    
    CONV2_SIZE = 5
    CONV2_DEEP = 64
    
    FC1_SIZE = 128
    FC2_SIZE = 64
    
    # 第一层卷积层
    with tf.variable_scope("layer1-conv1"):
#         conv1_weights = tf.get_variable(
#             "weight", [CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
#             initializer=tf.truncated_normal_initializer(stddev=0.1))
#          conv1_biases = tf.get_variable(
#             "bias", [CONV1_DEEP], initializer=tf.constant_initalizer(0.0))
        depth = int(list(image_batch.get_shape())[-1])
        print("depth with conv1:", depth)
        conv1_weights = tf.Variable(
            tf.truncated_normal([CONV1_SIZE, CONV1_SIZE, depth, CONV1_DEEP], stddev=0.1))
        conv1_biases = tf.Variable(tf.zeros(CONV1_DEEP))
        
        # 滤波器边长为5，深度为32，步长2，使用全0填充
        conv1 = tf.nn.conv2d(
            image_batch, conv1_weights, strides=[1,2,2,1], padding="SAME")
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))
        
        print("relu1's size:", relu1)
        
    # 第二层最大池化层, 滤波器边长2，全0填充，步长2
    with tf.name_scope('layer2-pool1'):
        pool1 = tf.nn.max_pool(
            relu1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        print("the size of pool1:", pool1)
        
    # 第三层 卷积层
    with tf.name_scope("layer3-conv2"):
#         conv2_weights = tf.get_variable(
#             "weight", [CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
#             intializer=tf.truncated.normal_initializer(stddev=0.1))
#         conv2_biases = tf.get_variable(
#             "bias", [CONV2_DEEP], initializer=tf.constant_initializer(0.0))
        conv2_weights = tf.Variable(
            tf.truncated_normal([CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP], stddev=0.1))
        conv2_biases = tf.Variable(tf.zeros(CONV2_DEEP))
        # 滤波器边长为5， 深度64， 步长1， 全0填充
        conv2 = tf.nn.conv2d(
            pool1, conv2_weights, strides=[1,2,2,1], padding="SAME")
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
        
        print("the size of relu2:", relu2)
    # 第四层 最大池化层
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(
            relu2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        print("the size of pool2:", pool2)
    
    # 扁平化层
    with tf.name_scope("layer5-flatten"):
        flatten = tf.layers.Flatten()(pool2)
        print("the size of flatten:", flatten)
        
    # 全连接层
    with tf.name_scope("layer6-fc1"):
        fc1 = tf.contrib.layers.fully_connected(flatten, FC1_SIZE)
        if train: fc1 = tf.nn.dropout(fc1, 0.5)
        print("the size of fc1:", fc1)
    
    # 全连接层
    with tf.name_scope("layer7-fc2"):
        fc2 = tf.contrib.layers.fully_connected(fc1, FC2_SIZE)
        print("the size of fc2:", fc2)
        
    #输出层
    with tf.name_scope("layer8-output"):
#         depth = list(fc2.get_shape())[1]
#         print(depth)
        output_weights = tf.Variable(tf.truncated_normal([FC2_SIZE, NUM_LABELS], stddev=0.1))
        output_biases = tf.Variable(tf.zeros(NUM_LABELS))
        
        logit = tf.add(tf.matmul(fc2, output_weights), output_biases)
        print("the size of logit:", logit)
    
    return logit

In [4]:
import tensorflow as tf
import numpy as np
import os
TRAIN_PATH = "C:/Users/yeyiy/Downloads/Dogs vs Cats/maybe"
TEST_PATH = "C:/Users/yeyiy/Downloads/Dogs vs Cats/maybe"
train_logs_dir = "./logs/train"
valid_logs_dir = "./logs/valid"

N_CLASSES = 2
weight = 299
height = 299
channels = 3
batch_size = 64
max_step = 10000
learning_rate = 0.01

train_images, train_labels, valid_images, valid_labels = \
    create_images_list(TRAIN_PATH)
train_image_batch, train_label_batch = make_batch(
    train_images, train_labels, weight, height, batch_size)

valid_image_batch, valid_label_batch = make_batch(
    valid_images, valid_labels, weight, height, batch_size)

x = tf.placeholder(tf.float32, shape=[batch_size, weight, height, channels], name='x')
y = tf.placeholder(tf.int64, shape=[batch_size], name='y')

logits = inference(x, train=True)

print("train_label_batch:", train_label_batch)
print("logits:", logits)

# loss and optimizer
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
#     logits=logits, labels=train_label_batch))
with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits\
                        (logits=logits, labels=y)
        loss = tf.reduce_mean(cross_entropy, name='loss')
        tf.summary.scalar(scope.name+'/loss', loss)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

#Accuracy
# correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
correct_pred = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')



with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(train_logs_dir, sess.graph)
    valid_writer = tf.summary.FileWriter(valid_logs_dir, sess.graph)
    
    try:
        for step in np.arange(max_step):
            if coord.should_stop():
                break
            train_images, train_labels = sess.run(([train_image_batch, train_label_batch]))
            _, train_loss, train_acc = sess.run([train_step, loss, accuracy], feed_dict={
                                            x: train_images, y:train_labels})
            if step % 50 == 0:
                print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, train_loss, train_acc*100.0))
#                 summary_str = sess.run(summary)
#                 train_writer.add_summary(summary_str, step)

            if step % 200 == 0 or (step + 1) == max_step:
                valid_images, valid_labels = sess.run([valid_image_batch, valid_label_batch])
                valid_loss, valid_acc = sess.run([loss, accuracy],
                                                 feed_dict={x:valid_images, y:valid_labels})
                print('**  Step %d, valid loss = %.2f, valid accuracy = %.2f%%  **' %(step, valid_loss, valid_acc*100.0))
#                 summary_str = sess.run(summary)
#                 val_writer.add_summary(summary_str, step)

            if step % 2000 == 0 or (step + 1) == max_step:
                checkpoint_path = os.path.join(train_logs_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)
            
    except tf.errors.OutOfRangeError:
        print("Done!")
    finally:
        coord.request_stop()
    coord.join(threads)


C:\Users\yeyiy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


laebls: [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
num_images: 16
input_data: 1
input_labels: [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
input_images: C:/Users/yeyiy/Downloads/Dogs vs Cats/maybe\cat.5.jpg
images: Tensor("div:0", shape=(299, 299, 3), dtype=float32)
labels: Tensor("input_producer/Gather_1:0", shape=(), dtype=int32)
images: Tensor("div_1:0", shape=(299, 299, 3), dtype=float32)
labels: Tensor("input_producer_1/Gather_1:0", shape=(), dtype=int32)
depth with conv1: 3
relu1's size: Tensor("layer1-conv1/Relu:0", shape=(64, 150, 150, 32), dtype=float32)
the size of pool1: Tensor("layer2-pool1/MaxPool:0", shape=(64, 75, 75, 32), dtype=float32)
the size of relu2: Tensor("layer3-conv2/Relu:0", shape=(64, 38, 38, 64), dtype=float32)
the size of pool2: Tensor("layer4-pool2/MaxPool:0", shape=(64, 19, 19, 64), dtype=float32)
the size of flatten: Tensor("layer5-flatten/flatten/Reshape:0", shape=(64, 23104), dtype=float32)
the size of fc1: Tensor("layer6-fc1/dropout/mul:0", shape=(64, 128)

KeyboardInterrupt: 